In [ ]:
from pymongo import MongoClient
import numpy as np
import utils.const as const

In [ ]:
client = MongoClient()
myCollect=client[db][collection]

In [ ]:
db=const.grpId
collection=const.PostCollection


outFile='data.csv'
outFileMini='dataMini.csv'
inFileI='vgg16_post.csv'
inFileT='text_post.csv'
textVectors={}
done={}

miniSize=2500
textVectorLen=100
imageVectorLen=7*7*512

In [ ]:
with open(inFileI,'r') as fp:
  for k in fp:
    print(k[:33],k[34:37],k[-2:]+'=')
    break

In [ ]:
with open(inFileT,'r') as fp:
  for k in fp:
    if len(k)<33:
      continue

    textVectors[k[:33]]=k[34:-1]

In [ ]:
def getNull(l=0):
  ans=""
  for _ in range(l):
    ans+=','
  return ans


In [ ]:
list1=['post_id','lang_en','lang_gu','lang_hi','created_time','updated_time','len_message','img_count','vid_count']
list2=['reactions','comment_sentiment','comment_count']

In [ ]:
def getDataFromDB(post_id):
  ans={}
  ans['post_id']=post_id
  
  for post in myCollect.find({'_id':post_id}):
    ans['lang_en']=str(0)
    ans['lang_gu']=str(0)
    ans['lang_hi']=str(0)

    if 'lang' in post:
      l = str(post['lang'])
      if l=='en':
        ans['lang_en']=str(1)
      elif l=='gu':
        ans['lang_gu']=str(1)
      elif l=='hi':
        ans['lang_hi']=str(1)

    if 'created_time' in post:
      ans['created_time']=str(post['created_time'])
    else:
      ans['created_time']=getNull()

    if 'updated_time' in post:
      ans['updated_time']=str(post['updated_time'])
    else:
      ans['updated_time']=getNull()

    if 'message' in post:
      ans['len_message']=str(len(post['message']))
    else:
      ans['len_message']=str(0)

    if 'attachments' in post and 'images' in post['attachments']:
      ans['img_count']=str(len(post['attachments']['images']))
    else:
      ans['img_count']=str(0)

    if 'attachments' in post and 'videos' in post['attachments']:
      ans['vid_count']=str(len(post['attachments']['videos']))
    else:
      ans['vid_count']=str(0)

    if 'reactions' in post:
      ans['reactions']=str(post['reactions'])
    else:
      ans['reactions']=getNull()

    if 'comment_sentiment' in post:
      ans['comment_sentiment']=str(post['comment_sentiment'])
    else:
      ans['comment_sentiment']=getNull()

    if 'comments' in post:
      ans['comment_count']=str(len(post['comments']))
    else:
      ans['comment_count']=str(0)
    
    break
  
  return ans

In [ ]:
def getTextVector(post_id):
  if post_id in textVectors:
    return textVectors[post_id]
  else:
    return getNull(imageVectorLen)

In [ ]:
fp=open(outFile,'w')
s=''
for k in list1:
  s+=k+','
for i in range(textVectorLen):
  s+='t'+str(i+1)+','
for i in range(imageVectorLen):
  s+='i'+str(i+1)+','
for k in list2:
  s+=k+','
fp.write(s[:-1]+'\n')

In [ ]:
done={}
with open(inFileI,'r') as f:
  for line in f:
    post_id=line[:33]
    image_vector=line[34:-1]
    text_vector=getTextVector(post_id)
    data=getDataFromDB(post_id)

    if post_id in done:
      print('bang')

    s=''

    for k in list1:
      s+=data[k]+','
    s+=text_vector+','
    s+=image_vector+','
    for k in list2:
      s+=data[k]+','
    
    fp.write(s[:-1]+'\n')

    done[post_id]=1

In [ ]:
for post in myCollect.find():
  if post['_id'] not in done:
    post_id=post['_id']
    image_vector=getNull(imageVectorLen)
    text_vector=getTextVector(post_id)
    data=getDataFromDB(post_id)
    s=''

    for k in list1:
      s+=data[k]+','
    s+=text_vector+','
    s+=image_vector+','
    for k in list2:
      s+=data[k]+','
    
    fp.write(s[:-1]+'\n')

    done[post_id]=1

In [ ]:
print(len(done))

In [ ]:
fp.close()

In [ ]:
dict1={}
for k in np.arange(len(done))[:miniSize]:
  dict1[k]=1

In [ ]:
fp=open(outFileMini,'w')
with open(outFile,'r') as f:
  i=-1
  for k in f:
    i+=1
    if i in dict1:
      fp.write(k)
fp.close()